# Sistema de Análisis Financiero Automatizado con IA
Mercedes González  
**(Business Case 6)**  

## 1: Imports e instalación
En la siguientes celdas instalo e importo las librerías necesarias.

In [3]:
# Instalación de la librería de Yahoo Finace
!pip install yfinance

In [4]:
# Importalación de la librería de Yahoo Finance, Pandas...
import yfinance as yf
import pandas as pd
import os
from datetime import datetime, timedelta

In [5]:
# Instalación de la librería de Cohere
!pip install cohere

In [6]:
# Importación de la librería Cohere y conexión
import cohere

# API Key - En producción, usar variables de entorno
API_KEY = "AX8kqTNN9sgvKyBTB39WPUQuzm4LJJVOiIi4JlRI" # mi propia API Key 
co = cohere.Client(API_KEY)

## 2: Definición de Parámetros
Defino el diccionario `params`que controla todo el análisis. 

Cambiando `params` se debe regenerar todo.

In [7]:
params = {
    'ticker': 'AAPL',
    'ticker_name': 'Apple Inc.',
    'start_date': '2025-01-01',
    'end_date': '2025-06-30',
    'benchmark': '^GSPC',
    'benchmark_name': 'S&P 500',
    # --- VARIABLES DE CONTEXTO EXTRA---
    'analyst_role': 'Analista financiero senior de Quantum Capital Partners', # Define la "persona"
    'audience': 'Comité de Inversiones',      # Define a quién le habla
    'tone': 'técnico, directo y centrado en la volatilidad' # Define el tono 
}

print(f"Configuración cargada. Modo analista: {params['analyst_role']}")

Configuración cargada. Modo analista: Analista financiero senior de Quantum Capital Partners


## 3: Descarga de Datos con Multi-Index Columns

In [8]:
def descargar_datos(ticker, inicio, fin):
    print(f"Descargando datos para {ticker}...")
    df = yf.download(ticker, start=inicio, end=fin, progress=False)
    
    # Limpieza de MultiIndex (según demo_yfinance página 3)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0) # Nos quedamos con el primer nivel (Close, Open, etc)
    
    df.reset_index(inplace=True) # Convertir la fecha en columna
    return df

# Descarga de ambos datasets
df_ticker = descargar_datos(params['ticker'], params['start_date'], params['end_date'])
df_benchmark = descargar_datos(params['benchmark'], params['start_date'], params['end_date'])

# Verificación rápida
print(f"\nDatos descargados: {len(df_ticker)} días para {params['ticker']} y {len(df_benchmark)} días para Benchmark.")
df_ticker.head()

Descargando datos para AAPL...


/var/tmp/ipykernel_7250/2525506664.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=inicio, end=fin, progress=False)


Descargando datos para ^GSPC...


/var/tmp/ipykernel_7250/2525506664.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=inicio, end=fin, progress=False)



Datos descargados: 121 días para AAPL y 121 días para Benchmark.


Price,Date,Close,High,Low,Open,Volume
0,2025-01-02,242.752090,247.978453,240.731232,247.809205,55740700
1,2025-01-03,242.264297,243.080598,240.800915,242.264297,40244100
2,2025-01-06,243.896912,246.216423,242.105013,243.210016,45045600
3,2025-01-07,241.119476,244.444435,240.263348,241.885999,40856000
4,2025-01-08,241.607269,242.612732,238.969207,240.830782,37628900


## 4: Cálculo de Métricas 
Se piden 3 grupos de métricas. Hay que calcularlas todas automáticamente.

In [9]:
# --- Grupo 1: KPIs Básicos ---
precio_inicial = df_ticker['Close'].iloc[0]
precio_final = df_ticker['Close'].iloc[-1]
precio_max = df_ticker['Close'].max()
precio_min = df_ticker['Close'].min()
cambio_dinero = precio_final - precio_inicial
cambio_porcentual = ((precio_final - precio_inicial) / precio_inicial) * 100
vol_promedio = df_ticker['Volume'].mean()
vol_total = df_ticker['Volume'].sum()

# --- Grupo 2: Análisis de Días ---
# Cálculo del cambio porcentual diario
df_ticker['Daily_Return'] = df_ticker['Close'].pct_change() * 100

dias_positivos = (df_ticker['Daily_Return'] > 0).sum()
dias_negativos = (df_ticker['Daily_Return'] < 0).sum()
dias_sin_cambio = (df_ticker['Daily_Return'] == 0).sum()

mejor_dia_val = df_ticker['Daily_Return'].max()
mejor_dia_fecha = df_ticker.loc[df_ticker['Daily_Return'].idxmax(), 'Date'].strftime('%Y-%m-%d')

peor_dia_val = df_ticker['Daily_Return'].min()
peor_dia_fecha = df_ticker.loc[df_ticker['Daily_Return'].idxmin(), 'Date'].strftime('%Y-%m-%d')

# --- Grupo 3: Comparación Benchmark ---
bench_inicial = df_benchmark['Close'].iloc[0]
bench_final = df_benchmark['Close'].iloc[-1]
cambio_benchmark = ((bench_final - bench_inicial) / bench_inicial) * 100

diferencia_vs_benchmark = cambio_porcentual - cambio_benchmark

if diferencia_vs_benchmark > 0:
    desempeno_relativo = "SUPERÓ"
else:
    desempeno_relativo = "NO SUPERÓ"
    
##-----CREACIÓN DE DATAFRAMES EXTRA----

# 1. Top 3 de mejores días (para ver patrones de subida)
top_gains = df_ticker.nlargest(3, 'Daily_Return')[['Date', 'Close', 'Daily_Return', 'Volume']]
# Formateamos la fecha para que ocupe menos espacio visual
top_gains['Date'] = top_gains['Date'].dt.strftime('%Y-%m-%d')

# 2.  Top 3 de peores días (para análisis de riesgo)
top_losses = df_ticker.nsmallest(3, 'Daily_Return')[['Date', 'Close', 'Daily_Return', 'Volume']]
top_losses['Date'] = top_losses['Date'].dt.strftime('%Y-%m-%d')

# Conversión a string 
table_gains_str = top_gains.to_string(index=False)
table_losses_str = top_losses.to_string(index=False)

print("✅ Todas las métricas calculadas exitosamente.")
print("✅ Dataframes extra generados para inyección en prompt.")

✅ Todas las métricas calculadas exitosamente.
✅ Dataframes extra generados para inyección en prompt.


## 5: Construcción del Prompt para la IA

In [10]:
prompt = f"""
Rol: Actúa como un {params['analyst_role']}.
Audiencia: Escribes para el {params['audience']}.
Tono: {params['tone']}.

TAREA:
Genera un Resumen Ejecutivo Financiero analizando el desempeño de {params['ticker_name']} ({params['ticker']}) contra el {params['benchmark_name']}.

--- DATOS GENERALES ---
Período: {params['start_date']} a {params['end_date']}
Precio Inicio: ${precio_inicial:.2f} | Cierre: ${precio_final:.2f}
Rendimiento Total: {cambio_porcentual:.2f}% (vs Benchmark: {cambio_benchmark:.2f}%)
Resultado Relativo: {desempeno_relativo} por {abs(diferencia_vs_benchmark):.2f}%

--- ANÁLISIS DE VOLATILIDAD (DATAFRAMES INYECTADOS) ---
No te limites a mencionar el mejor/peor día. Analiza las siguientes tablas de movimientos extremos para detectar patrones:

TABLA A: TOP 3 DÍAS POSITIVOS
{table_gains_str}

TABLA B: TOP 3 DÍAS NEGATIVOS
{table_losses_str}

--- INSTRUCCIONES DE REDACCIÓN ---
1. Empienza con análisis coherente de 1 párrafo. No solo listes los datos, interprétalos.
2. Sigue con una conclusión contundente sobre si el activo es seguro o arriesgado comparado con el {params['benchmark_name']}.
2. Utiliza los datos de las Tablas A y B para explicar la volatilidad. Por ejemplo, si hubo mucho volumen en las caídas, menciónalo como señal de venta fuerte.
3. Mantén el tono {params['tone']} solicitado.
"""

print("Prompt Avanzado Generado.")

Prompt Avanzado Generado.


In [11]:
print(prompt)


Rol: Actúa como un Analista financiero senior de Quantum Capital Partners.
Audiencia: Escribes para el Comité de Inversiones.
Tono: técnico, directo y centrado en la volatilidad.

TAREA:
Genera un Resumen Ejecutivo Financiero analizando el desempeño de Apple Inc. (AAPL) contra el S&P 500.

--- DATOS GENERALES ---
Período: 2025-01-01 a 2025-06-30
Precio Inicio: $242.75 | Cierre: $200.66
Rendimiento Total: -17.34% (vs Benchmark: 5.19%)
Resultado Relativo: NO SUPERÓ por 22.53%

--- ANÁLISIS DE VOLATILIDAD (DATAFRAMES INYECTADOS) ---
No te limites a mencionar el mejor/peor día. Analiza las siguientes tablas de movimientos extremos para detectar patrones:

TABLA A: TOP 3 DÍAS POSITIVOS
      Date      Close  Daily_Return    Volume
2025-04-09 198.172348     15.328841 184395900
2025-05-12 210.347122      6.314615  63775800
2025-04-11 197.474731      4.059449  87435900

TABLA B: TOP 3 DÍAS NEGATIVOS
      Date      Close  Daily_Return    Volume
2025-04-03 202.497559     -9.245611 103419000
20

## 6: Llamada a la API de Cohere para generar Análisis con IA

In [12]:
print("Generando análisis con IA... (esto puede tardar unos segundos)")

try:
    response = co.chat(
        message=prompt,
        model="command-a-03-2025", 
        temperature=0.1
    )
    resumen_ia = response.text
    print("\n=== RESUMEN EJECUTIVO ===\n")
    print(resumen_ia)
except Exception as e:
    print(f"Error en la llamada a la IA: {e}")
    resumen_ia = "Error al generar el resumen."

Generando análisis con IA... (esto puede tardar unos segundos)

=== RESUMEN EJECUTIVO ===

**Resumen Ejecutivo Financiero: Desempeño de Apple Inc. (AAPL) vs. S&P 500 (2025-01-01 a 2025-06-30)**  

Durante el primer semestre de 2025, Apple Inc. (AAPL) registró un rendimiento total de **-17.34%**, significativamente por debajo del S&P 500, que avanzó **5.19%**. Este resultado relativo de **-22.53%** refleja un desempeño atípicamente débil para un componente clave del índice. El precio de la acción cayó desde $242.75 a $200.66, con una volatilidad marcada por movimientos extremos concentrados en abril. El análisis de los días con mayores fluctuaciones revela un patrón de volatilidad asimétrica: las caídas fueron más abruptas y acompañadas de volúmenes elevados, mientras que las recuperaciones mostraron menor convicción en términos de volumen. Este comportamiento sugiere una mayor sensibilidad de AAPL a noticias negativas o ajustes de riesgo sistémico, amplificando su subdesempeño frente a

## 7: Exportación y Organización de Archivos
Creación de las carpetas donde guardaremos los csv y propmt.

In [13]:
# Creación del nombre de la carpeta con timestamp
timestamp = datetime.now().strftime("%Y%m%d")
# Usamos el nombre del ticker desde tus params
folder_name = f"output/{timestamp}_analisis_{params['ticker']}"
data_folder = f"{folder_name}/datos"

# Creación de directorios
# os.makedirs crea las carpetas necesarias si no existen
os.makedirs(data_folder, exist_ok=True)

# Guardar CSVs
# Asumiendo que df_ticker y df_benchmark ya existen en memoria
df_ticker.to_csv(f"{data_folder}/datos_historicos.csv", index=False)
df_benchmark.to_csv(f"{data_folder}/benchmark_datos.csv", index=False)

# Guardar Prompt
# Asumiendo que la variable 'prompt' ya fue creada en la celda anterior
with open(f"{data_folder}/prompt_usado.txt", "w", encoding="utf-8") as f:
    f.write(prompt)

# Guardar Resumen IA (si ya generaste la variable resumen_ia)
if 'resumen_ia' in locals():
    with open(f"{folder_name}/resumen_ejecutivo.txt", "w", encoding="utf-8") as f:
        f.write(resumen_ia)

print(f"✅ ¡Proceso completado! Archivos guardados en: {folder_name}")

✅ ¡Proceso completado! Archivos guardados en: output/20251211_analisis_AAPL
